In [32]:
## Requirement
import numpy as np
import pandas as pd
import sklearn
from sklearn import model_selection
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import RandomForestRegressor
import sklearn.preprocessing as sp
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
import xgboost as xgb
import lightgbm as lgb

def meanAbsoluteError(y_test, y_pred):
    """for val"""
    loss = 0
    for i in range(len(y_test)):
        loss += abs(y_test[i] - y_pred[i])
    loss /= len(y_test)
    return loss


In [33]:
# Train・Testデータ抽出
df_train= pd.read_csv("./dataset/pubg-train.csv")
df_test = pd.read_csv("./dataset/pubg-test.csv")

def get_more_train(df_train, df_test):

    df_test["matchgroupId"] = df_test["matchId"]+df_test["groupId"]
    #[winPlacePerc, Id, matchId, groupId]
    df_train = df_train[["winPlacePerc","matchId", "groupId"]].sort_values(by=["matchId", "groupId"])
    df_train["matchgroupId"] = df_train["matchId"] + df_train["groupId"]
    df_train = df_train.drop(columns=["matchId", "groupId"])
    df_train = df_train.groupby("matchgroupId").mean()
    
    df_merge = pd.merge(df_test, df_train , on="matchgroupId")
    df_merge = df_merge.sort_values(by=["matchgroupId"])
    df_merge = df_merge[["Id", "winPlacePerc"]]
    df_merge2 = pd.merge(df_test, df_merge, on="Id", how="left")
    df_merge2 = df_merge2.drop(columns="matchgroupId")
    df_merge2 = df_merge2.dropna(how="any")
    df_test = df_test.drop(columns="matchgroupId")
    return df_merge2

In [34]:
get_more_train(df_train, df_test).dtypes

Id                  object
groupId             object
matchId             object
assists              int64
boosts               int64
damageDealt        float64
DBNOs                int64
headshotKills        int64
heals                int64
killPlace            int64
killPoints           int64
kills                int64
killStreaks          int64
longestKill        float64
matchDuration        int64
matchType           object
numGroups            int64
rankPoints           int64
revives              int64
rideDistance       float64
roadKills            int64
swimDistance       float64
teamKills            int64
vehicleDestroys      int64
walkDistance       float64
weaponsAcquired      int64
winPoints            int64
winPlacePerc       float64
dtype: object

In [35]:
# Train・Testデータ抽出
#df_train= pd.read_csv("./dataset/pubg-train.csv")
#df_test = pd.read_csv("./dataset/pubg-test.csv")

def drop_and_add(df_train, df_test):
    ep = 0.00001
    col_to_drop_from_mp = ["killPoints", "rankPoints"]
    df = pd.concat([df_train, df_test])
    #df = df.drop(columns=["Id"])
    
    df['_killPoints_rankpoints'] = df['rankPoints']+df['killPoints']
    df["_totalDistance"] = 0.25*df["rideDistance"]+df["walkDistance"]+df["swimDistance"]
    df['_walkDistance_heals_Ratio'] = df['walkDistance'] / df['heals']
    df['_walkDistance_kills_Ratio'] = df['walkDistance'] / df['kills']
    df['_kills_walkDistance_Ratio'] = df['kills'] / df['walkDistance']
    df['_totalDistancePerDuration'] =  df["_totalDistance"]/df["matchDuration"]
    df['_killPlace_kills_Ratio'] = df['killPlace']/df['kills']
    df['_walkDistancePerDuration'] =  df["walkDistance"]/df["matchDuration"]
    df['_walkDistancePerc'] = df.groupby('matchId')['walkDistance'].rank(pct=True).values
    df['_killPerc'] = df.groupby('matchId')['kills'].rank(pct=True).values
    df['_killPlacePerc'] = df.groupby('matchId')['killPlace'].rank(pct=True).values
    df['_weaponsAcquiredPerc'] = df.groupby('matchId')['weaponsAcquired'].rank(pct=True).values
    df['_walkDistance_kills_Ratio2'] = df['_walkDistancePerc'] / df['_killPerc']
    df['_kill_kills_Ratio2'] = df['_killPerc']/df['_walkDistancePerc']
    df['_killPlace_walkDistance_Ratio2'] = df['_walkDistancePerc']/df['_killPlacePerc']
    df['_killPlace_kills_Ratio2'] = df['_killPlacePerc']/df['_killPerc']
    df['_totalDistancePerc'] = df.groupby('matchId')['_totalDistance'].rank(pct=True).values
    df['_walkDistance_kills_Ratio3'] = df['_walkDistancePerc'] / df['kills']
    df['_walkDistance_kills_Ratio4'] = df['kills'] / df['_walkDistancePerc']
    df['_walkDistance_kills_Ratio5'] = df['_killPerc'] / df['walkDistance']
    df['_walkDistance_kills_Ratio6'] = df['walkDistance'] / df['_killPerc']
    df.loc[df["rankPoints"]==-1, "rankPoints"] =0
    df["_headshot_ratio"] = df["headshotKills"]/df["kills"]
    df["_kills_over_headshotkills"] = df["kills"]/df["headshotKills"]
    df["_killstreak_ratio"] = df["killStreaks"]/df["kills"]
    
    df["_weaponAcquired_per_distance"] = df["_totalDistancePerc"]/df["weaponsAcquired"]
    df["_weaponsAcquired_matchDuration"] = df["weaponsAcquired"]/df["matchDuration"]
    df["_kill_assists"] = df["kills"]*df["assists"]
    df["_healboosts"] = df["heals"]*df["boosts"]
    df["_skill"] = df["headshotKills"] * df["roadKills"]
    df["_killPlace"] = -df["killPlace"]
    df["_hideDuration"] = df["matchDuration"]-(df["walkDistance"]/6+df["rideDistance"]/277+df["swimDistance"])
    
    #df["_Zombie"] = 0
    #df[(df["kills"]==0) & (df["_totalDistance"]<50) & (df["weaponsAcquired"]==0)]["_Zombie"] = -1
    
    #カテゴリラベルをone-hot化
    le = sp.LabelEncoder()
    le.fit(df["matchType"].unique())
    df["matchType"] = le.fit_transform(df["matchType"])
    ohe = sp.OneHotEncoder()
    enced = ohe.fit_transform(df["matchType"].values.reshape(1, -1).transpose())
    temp = pd.DataFrame(index=df["matchType"].index, columns="matchType-" + le.classes_, data=enced.toarray())
    df = pd.concat([df, temp], axis=1)

    df = df.drop(columns=["matchType"])
    #df = df.drop(columns=["matchId", "groupId"])
    
    df_train = df[df["winPlacePerc"].notnull()].reset_index(drop=True)
    df_test = df[df["winPlacePerc"].isnull()].drop(["winPlacePerc"], axis=1).reset_index(drop=True)
    
    df_train[df_train == np.Inf] = np.NaN
    df_test[df_test == np.Inf] = np.NaN
    df_train[df_train == np.NINF] = np.NaN
    df_test[df_test == np.NINF] = np.NaN
    df_train.fillna(0, inplace=True)
    df_test.fillna(0, inplace=True)
    
    
    return df_train, df_test
    
def pre_process(df_train, df_test):
    """pandasから得たデータフレームの整形
    Arg:
        df_train(pd.DataFrame) 訓練用データフレーム
        df_test(pd.DataFrame) テスト用データフレーム
        
    Return:
        X_train(ndarray(float)) 訓練用の入力データ(加工済み)
        Y_train(ndarray(float)) 訓練用のラベルデータ(確率:0~1)
        X_test(ndarray(float)) テスト用の入力データ(X_trainと同じshape(-1))
    """
    
    df_train_more= get_more_train(df_train, df_test)
    df_train = pd.concat([df_train, df_train_more])
    print(df_train.shape)
    df_test = df_test.drop(columns="matchgroupId")
    df_train ,df_test = drop_and_add(df_train, df_test)
    df_train = df_train.drop(columns=["Id","matchId", "groupId"])
    df_test = df_test.drop(columns=["Id","matchId", "groupId"])
    Y_train = df_train["winPlacePerc"]
    X_train = df_train.drop(columns=["winPlacePerc"])
    X_test = df_test
    
    scaler_std = StandardScaler()
    scaler_std.fit(X_train)
 
    std_X_train = scaler_std.transform(X_train)
    std_X_test = scaler_std.transform(X_test)
    
    return X_train, Y_train, X_test

#df_train, df_test = drop_and_add(df_train, df_test)

#df_train.to_csv("_df_train.csv", index=False)
#df_test.to_csv("_df_test.csv", index=False)
#_df_train = pd.read_csv("_df_train.csv")
#_df_test = pd.read_csv("_df_test.csv")

df_train= pd.read_csv("./dataset/pubg-train.csv")
print(df_train.shape)
df_test = pd.read_csv("./dataset/pubg-test.csv")
X_train, Y_train, X_test = pre_process(df_train, df_test)
"""
Y_train = df_train["winPlacePerc"]
X_train = df_train.drop(columns="winPlacePerc")
X_test = pd.DataFrame(df_test)
"""

#X_train, Y_train, X_test = pre_process(df_train, df_test)
# train test split

#for validation
#X_train, X_test, Y_train, Y_testval = train_test_split(X_train, Y_train, test_size=0.1, random_state=0)

#X_train = pd.DataFrame(X_train)
#X_test = pd.DataFrame(X_test)

(356075, 28)
(423172, 28)


C:\Users\msy-o\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  
C:\Users\msy-o\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
C:\Users\msy-o\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:625: DataCon

'\nY_train = df_train["winPlacePerc"]\nX_train = df_train.drop(columns="winPlacePerc")\nX_test = pd.DataFrame(df_test)\n'

In [36]:
NFOLD = 3
SEED = 0

ntrain = X_train.shape[0]
ntest = X_test.shape[0]

kf = KFold(n_splits=NFOLD, shuffle=True, random_state=SEED)

In [37]:
#各学習器のラッパー
class SklearnWrapper(object):
    def __init__(self, clf, seed=0, params=None):
        params["random_state"] = seed
        self.clf = clf(**params)
        
    def train(self, x_train, y_train):
        self.clf.fit(x_train, y_train)
        
    def predict(self, x):
        return self.clf.predict(x) #proba??
    
class LightGBMWrapper(object):
    def __init__(self, clf, seed=0, params=None):
        params["feature_fraction_seed"] = seed
        params["bagging_seed"] = seed
        self.clf = clf(**params)
        
    def train(self, x_train, y_train):
        self.clf.fit(x_train, y_train)
        
    def predict(self, x):
        return self.clf.predict(x)
    
class XgbWrapper(object):
    def __init__(self, seed=0, params=None):
        self.params = params
        self.params["seed"] = seed
        self.nrounds = params.pop("nrounds", 250)
        
    def train(self, x_train, y_train):
        dtrain = xgb.DMatrix(x_train, label=y_train)
        self.gbdt = xgb.train(self.params, dtrain, self.nrounds)
    
    def predict(self, x):
        return self.gbdt.predict(xgb.DMatrix(x))

In [38]:
def get_oof(clf):
    oof_train = np.zeros((ntrain,))
    oof_test = np.zeros((ntest,))
    oof_test_skf = np.empty((NFOLD, ntest))
    
    for i, (train_index, test_index) in enumerate(kf.split(X_train)):
        X_tr = X_train.iloc[train_index, :]
        Y_tr = Y_train.iloc[train_index]
        X_te = X_train.iloc[test_index, :]
        
        clf.train(X_tr, Y_tr)
        
        oof_train[test_index] = clf.predict(X_te)#regのinput
        oof_test_skf[i, :] = clf.predict(X_test)
        
    oof_test[:] = oof_test_skf.mean(axis=0)
    return oof_train.reshape(-1, 1), oof_test.reshape(-1, 1)

In [39]:
et_params = {"n_jobs":16,
            "n_estimators":200,
            "max_features":0.7,
            "max_depth":12,
            "min_samples_leaf":2,}

rf_params = {"n_jobs":16,
            "n_estimators":200,
            "max_features":0.2,
            "max_depth":12,
            "min_samples_leaf":2,}

xgb_params = {'seed': 0,
              'colsample_bytree': 0.7,
              'silent': 1,
              'subsample': 0.7,
              'learning_rate': 0.075,
              'max_depth': 8,
              'num_parallel_tree': 5,
              'min_child_weight': 1,
              'nrounds': 500}

lightgbm_params1 = {'n_estimators':200,
                   'learning_rate':0.05,
                   'num_leaves':123,
                   'colsample_bytree':0.8,
                   'subsample':0.9,
                   'max_depth':15,
                   'reg_alpha':0.1,
                   'reg_lambda':0.1,
                   'min_split_gain':0.01,
                   'min_child_weight':2,
                   "seed":1}

lightgbm_params2 ={"n_estimators":200,
                  "learning_rate": 0.1,
                  "subsample":0.8,
                  "subsample_freq":4,
                  "colsample_bytree":0.3,
                  "num_leaves":150,
                  "seed":1000,
                  "max_depth":15,
                  "max_features":"sqrt"}

lightgbm_params3 = {"n_estimators": 500,
                  "learning_rate":0.2,
                  "subsample":0.8,
                  "subsample_freq":6,
                  "colsample_bytree":0.8,
                  "num_leaves":50,
                  "seed":1001,
                  "max_depth":12}

In [40]:
xg = XgbWrapper(seed=SEED, params=xgb_params)
et = SklearnWrapper(clf=ExtraTreesRegressor, seed=SEED, params=et_params)
rf = SklearnWrapper(clf=RandomForestRegressor, seed=SEED, params=rf_params)
lg1 = LightGBMWrapper(clf=lgb.LGBMRegressor, seed=SEED, params=lightgbm_params1)
lg2 = LightGBMWrapper(clf=lgb.LGBMRegressor, seed=SEED, params=lightgbm_params2)
lg3 = LightGBMWrapper(clf=lgb.LGBMRegressor, seed=SEED, params=lightgbm_params3)

In [41]:
lg1_oof_train, lg1_oof_test = get_oof(lg1)
print("LG1-MAE:{}".format(mean_absolute_error(Y_train, lg1_oof_train)))

"""
et_oof_train, et_oof_test = get_oof(et)
print("ET-MAE:{}".format(mean_absolute_error(Y_train, et_oof_train)))
xg_oof_train, xg_oof_test = get_oof(xg)
print("XG-MAE:{}".format(mean_absolute_error(Y_train, xg_oof_train)))
rf_oof_train, rf_oof_test = get_oof(rf)
print("RF-MAE:{}".format(mean_absolute_error(Y_train, rf_oof_train)))
"""

lg2_oof_train, lg2_oof_test = get_oof(lg2)
print("LG2-MAE:{}".format(mean_absolute_error(Y_train, lg2_oof_train)))


lg3_oof_train, lg3_oof_test = get_oof(lg3)
print("LG3-MAE:{}".format(mean_absolute_error(Y_train, lg3_oof_train)))


LG1-MAE:0.06948295825294716
LG2-MAE:0.0697829106222704
LG3-MAE:0.07030123009885167


In [2]:
x_train = np.concatenate((lg1_oof_train, lg2_oof_train, lg3_oof_train), axis=1)
x_test = np.concatenate((lg1_oof_test, lg2_oof_test, lg3_oof_test), axis=1)
#x_train = np.concatenate((lg1_oof_train, lg2_oof_train), axis=1)
#x_test = np.concatenate((lg1_oof_test, lg2_oof_test), axis=1)

# train test split
X_train, X_val, y_train, Y_val = train_test_split(x_train, Y_train, test_size=0.2, random_state=0)

print("{},{}".format(x_train.shape, x_test.shape))

lgb_train = lgb.Dataset(np.array(X_train),np.array(y_train))
lgb_eval = lgb.Dataset(np.array(X_val),np.array(Y_val),reference=lgb_train)

params={'learning_rate': 0.05,
        "n_estimators" : 10000,
        'objective':'poisson',
        'metric':'mae',
        'num_leaves': 31,
        'verbose': 1,
        'random_state':42,
        'bagging_fraction': 0.7,
        'feature_fraction': 0.7
       }


gbm = lgb.train(params,
                lgb_train,
                num_boost_round=7500,
                valid_sets=lgb_eval,
                early_stopping_rounds=30,
               )

def meanAbsoluteError(y_test, y_pred):
    loss = 0
    for i in range(len(y_test)):
        loss += abs(y_test[i] - y_pred[i])
    loss /= len(y_test)
    return loss

Y_pred= gbm.predict(np.array(x_test), num_iteration=gbm.best_iteration)

NameError: name 'np' is not defined

In [43]:
#print(meanAbsoluteError(Y_testval.values, Y_pred))

In [44]:
def after_process(Y_pred):
    """予測結果を提出用に整形し，csvで保存
    Arg:
        Y_pred_submit(ndarray(float))
    Return:
        None
    """
    df_submit = pd.DataFrame(Y_pred, columns=["winPlacePerc"])
    df_submit.loc[df_submit['winPlacePerc'] < 0, 'winPlacePerc'] = 0.0
    df_submit.loc[df_submit['winPlacePerc'] > 1.0, 'winPlacePerc'] = 1.0
    df_submit.to_csv("3predict_test.csv", index=False)
    print("保存done")
    return None

#after_process(Y_pred)

In [45]:
def add_match_group_data(Y_pred, df_test, df_train):
    """matchとgroupが一致するデータを統合"""
    df_pred = pd.DataFrame(Y_pred, columns=["winPlacePerc"])
    df_pred = pd.concat([df_pred, df_test[["Id", "matchId", "groupId",
                                           "kills", "_totalDistance", "weaponsAcquired"]]], axis=1)
    df_pred["matchgroupId"] = df_pred["matchId"]+df_pred["groupId"]
    df_pred = df_pred.drop(columns=["matchId", "groupId"])
    #Remove Zombi as 0
    df_pred[(df_pred["kills"]==0) & (df_pred["_totalDistance"]<30) & (df_pred["weaponsAcquired"]==0)]["winPlacePerc"]=0.025
    
    
    #[winPlacePerc, Id, matchId, groupId]
    df_train = df_train[["winPlacePerc","matchId", "groupId"]].sort_values(by=["matchId", "groupId"])
    df_train["matchgroupId"] = df_train["matchId"] + df_train["groupId"]
    df_train = df_train.drop(columns=["matchId", "groupId"])
    df_train = df_train.rename(columns={"winPlacePerc":"wpp_acc"})
    df_train = df_train.groupby("matchgroupId").mean()
    
    df_merge = pd.merge(df_pred, df_train , on="matchgroupId")
    df_merge = df_merge.sort_values(by=["matchgroupId"])
    df_merge = df_merge[["Id", "wpp_acc"]]
    df_merge2 = pd.merge(df_pred, df_merge, on="Id", how="left")
    
    df_submit = df_merge2["wpp_acc"]
    df_pred = df_pred["winPlacePerc"]
    print(df_pred.shape)
    
    df_submit = df_submit.values
    print(df_submit.shape)
    df_pred = df_pred.values
    
    
    for i in range(len(df_submit)):
        if np.isnan(df_submit[i]):
            df_submit[i] = df_pred[i]

    
    #delete Zombie
    
    
    
    #print(df_submit)
    
    """
    matchgroupId2wpp = {}
    for i in range(df_merge.shape[0]):
        matchgroundId = df_merge["matchgroupId"][i]
        wpp_acc = df_merge["wpp_acc"][i]
        matchgroupId2wpp[matchgroundId] = wpp_acc
    """
    
    return df_submit

In [46]:
df_train= pd.read_csv("./dataset/pubg-train.csv")
df_test = pd.read_csv("./dataset/pubg-test.csv")
df_test["_totalDistance"] = 0.25*df_test["rideDistance"]+df_test["walkDistance"]+df_test["swimDistance"]
Y_submit = add_match_group_data(Y_pred, df_test, df_train)
after_process(Y_submit)

C:\Users\msy-o\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


(89019,)
(89019,)
保存done


In [47]:
df_sota = pd.read_csv("now_sota0487.csv")
Y_sota = df_sota.values
print(meanAbsoluteError(Y_sota, Y_submit))

[0.00469146]
